In [1]:
import pandas as pd

In [2]:
excel_schedule=pd.read_excel('new_test_excel.xlsx')

In [3]:
df = excel_schedule.copy()
df =  df.drop(df.columns[0], axis=1)
df = df.drop([0,1], axis=0)
df = df.reset_index()
df =  df.drop(df.columns[0], axis=1)
df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Время,Даты,Название,Места проведения,Преподаватели
1,09:00–10:30,24.05,Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
2,09:00–10:30,22.02\n15.03–22.03 (2)\n05.04\n19.04\n03.05\n1...,Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
3,10:45–12:15,15.02–01.03 (3)\n15.03–03.05 (8)\n17.05–24.05 (2),Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
4,13:15–14:45,15.02–01.03 (3)\n15.03–03.05 (8)\n17.05–24.05 (2),"Маркетинг, лекция",С использованием информационно-коммуникационны...,Молчанов Н. Н.


In [4]:
column_name = ['Time', 'Date','SubjectName', 'Place', 'Teacher']
df = df.set_axis((column_name), axis=1, inplace=False)
df = df.drop([0], axis=0)
df.head()

,Time,Date,SubjectName,Place,Teacher
1,09:00–10:30,24.05,Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
2,09:00–10:30,22.02\n15.03–22.03 (2)\n05.04\n19.04\n03.05\n1...,Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
3,10:45–12:15,15.02–01.03 (3)\n15.03–03.05 (8)\n17.05–24.05 (2),Теория вероятностей и математическая статистик...,С использованием информационно-коммуникационны...,Ананьевский С. М.
4,13:15–14:45,15.02–01.03 (3)\n15.03–03.05 (8)\n17.05–24.05 (2),"Маркетинг, лекция",С использованием информационно-коммуникационны...,Молчанов Н. Н.
5,15:00–16:30,15.02–01.03 (3)\n15.03–29.03 (3),"Электив. Интернет-аналитика для экономиста, ле...",С использованием информационно-коммуникационны...,Юрков А. В.


In [5]:
def iter_over_colmns(df, max_n):
    return [df[i] for i in range(max_n)]

def parse_date(x):
    if x[-1]==')':
        dates, repeate_time = x.split('(')
        date_start, date_end = dates.strip().split('–')
        repeate_time = repeate_time[:-1]
    else:
        date_start = x
        date_end = x
        repeate_time = 1
    return( date_start, date_end, repeate_time)

In [6]:
parse_date('12.02–30.04 (12)')

('12.02', '30.04', '12')

In [7]:
df = df.assign(Date=df.Date.str.split('\n').explode('Date'))

In [8]:
df_test=df.copy()
# df_test

In [9]:
df_parsed_data = df['Date'].apply(lambda x: pd.Series(parse_date(x)))
df['StartDate'], df['EndDate'], df['RepeateTime'] = iter_over_colmns(df_parsed_data, 3)
df.tail()

,Time,Date,SubjectName,Place,Teacher,StartDate,EndDate,RepeateTime
96,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12
97,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12
98,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12
99,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12
100,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12


In [11]:
def find_sub_group(subject: str):
    separation='_x000d_\n'
    if subject.find(separation) != -1:
        subject_name, sub_group = subject.split(separation)
    else:
        subject_name, sub_group= subject, ''
    return subject_name, sub_group
# find_sub_group('Траектория 3 (В1 – В2). Английский язык, практическое занятие_x000d_\nПодгруппа 7')

In [12]:
df_found_sub_group = df['SubjectName'].apply(lambda x: pd.Series(find_sub_group(x)))
df['SubjectName'], df['SubGroup'] = iter_over_colmns(df_found_sub_group, 2)

In [13]:
def parse_time(time_str):
    def get_datetime_format(time_str:str):
        return pd.to_datetime(time_str, format='%H:%M').time()
    return list(map(get_datetime_format, time_str.split('–')))

In [14]:
df_parsed_time = df['Time'].apply(lambda x: pd.Series(parse_time(x)))
df['StartTime'], df['EndTime'] = iter_over_colmns(df_parsed_time, 2)

In [15]:
df.tail()

,Time,Date,SubjectName,Place,Teacher,StartDate,EndDate,RepeateTime,SubGroup,StartTime,EndTime
96,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12,Подгруппа 1,13:15:00,14:45:00
97,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12,Подгруппа 2,13:15:00,14:45:00
98,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12,,13:15:00,14:45:00
99,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12,Подгруппа 2,13:15:00,14:45:00
100,13:15–14:45,12.02–30.04 (12),"Вычислительные системы, сети и телекоммуникаци...",С использованием информационно-коммуникационны...,Юрков А. В.,12.02,30.04,12,Подгруппа 1,13:15:00,14:45:00


In [23]:
event_example = df.iloc[96]
# pd.to_pickle(event_example, "./dummy.pkl")
event_example.to_json(orient='index')

'{"Time":"13:15\\u201314:45","Date":"12.02\\u201330.04 (12)","SubjectName":"\\u0412\\u044b\\u0447\\u0438\\u0441\\u043b\\u0438\\u0442\\u0435\\u043b\\u044c\\u043d\\u044b\\u0435 \\u0441\\u0438\\u0441\\u0442\\u0435\\u043c\\u044b, \\u0441\\u0435\\u0442\\u0438 \\u0438 \\u0442\\u0435\\u043b\\u0435\\u043a\\u043e\\u043c\\u043c\\u0443\\u043d\\u0438\\u043a\\u0430\\u0446\\u0438\\u0438, \\u043a\\u043e\\u043d\\u0442\\u0440\\u043e\\u043b\\u044c\\u043d\\u0430\\u044f \\u0440\\u0430\\u0431\\u043e\\u0442\\u0430","Place":"\\u0421 \\u0438\\u0441\\u043f\\u043e\\u043b\\u044c\\u0437\\u043e\\u0432\\u0430\\u043d\\u0438\\u0435\\u043c \\u0438\\u043d\\u0444\\u043e\\u0440\\u043c\\u0430\\u0446\\u0438\\u043e\\u043d\\u043d\\u043e-\\u043a\\u043e\\u043c\\u043c\\u0443\\u043d\\u0438\\u043a\\u0430\\u0446\\u0438\\u043e\\u043d\\u043d\\u044b\\u0445 \\u0442\\u0435\\u0445\\u043d\\u043e\\u043b\\u043e\\u0433\\u0438\\u0439","Teacher":"\\u042e\\u0440\\u043a\\u043e\\u0432 \\u0410. \\u0412.","StartDate":"12.02","EndDate":"30.04","Rep